In [1]:
import numpy as np
DTYPE = np.float64
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import os
import pickle
import gzip
from tqdm import tqdm
import sys
sys.path.insert(0, "../../packages")
import flory_

In [2]:
# Flory Huggins Free Energy function
def floryHuggins(phi:DTYPE, chi:np.array):
    part_1 = np.sum(phi*np.log(phi))
    part_2 = 0

    for i in range(len(phi)):
        for j in range(i+1, len(phi)):
            part_2 += chi[i][j]*phi[i]*phi[j]

    return part_1 + part_2

In [3]:
# General Function for computing mergers of compartments
# Returns the concentrations of components in the merged and the unmerged compartments
# Stores them in phi_in_kmerged and phi_in_kunmerged respectively.


def mergers(concs:np.array, vols:np.array, chis, merged_compartments:list):
    # Find the unmerged compartment(s)
    expected = len(vols)*(len(vols)+1)//2
    actual = np.sum(merged_compartments)
    unmerged_compartment = expected - actual

    # print(concs)
    # print(vols)
    # print(unmerged)

    # Compute the merged volumes, stored in variable eta_merged
    # subtract -1 from the merged_compartments idxs to maintain python idxing
    eta_merged = 0
    for compartment in merged_compartments:
        eta_merged += vols[compartment-1]
    # print(eta_merged)

    # Calculating the compositions of components in the merged compartment
    # Taking a simple weighted average
    phi_1merged = (vols[merged_compartments[0]-1]*concs[0, merged_compartments[0]-1] + vols[merged_compartments[1]-1]*concs[0, merged_compartments[1]-1])/eta_merged
    phi_2merged = (vols[merged_compartments[0]-1]*concs[1, merged_compartments[0]-1] + vols[merged_compartments[1]-1]*concs[1, merged_compartments[1]-1])/eta_merged
    phi_3merged = 1 - phi_1merged - phi_2merged
    # print(phi_1merged, phi_2merged, phi_3merged)
    # print(phi_1merged + phi_2merged + phi_3merged)

    phi_in_kmerged = [phi_1merged, phi_2merged, phi_3merged]
    phi_in_kunmerged = [concs[0, unmerged_compartment-1], concs[1, unmerged_compartment-1], concs[2, unmerged_compartment-1]]
    # print(phi_in_kmerged)
    # print(phi_in_kunmerged)

    F_merged = eta_merged*floryHuggins(phi_in_kmerged, chis) + vols[unmerged_compartment-1]*floryHuggins(phi_in_kunmerged, chis)
    # print(F_merged)

    return phi_in_kunmerged, phi_in_kmerged, eta_merged, unmerged_compartment

    

In [4]:
# Global concentrations
phi_global = np.array([0.2, 0.3, 0.5], dtype = DTYPE)

# Xs = np.arange(0, 10.1, 0.1)
# Xs = [1]
# tol = DTYPE(1e-4)

In [5]:
chis = np.array([[0, 3.0, 3+1], [3.0, 0.0, 3.0], [3+1, 3.0, 0.0]], dtype = DTYPE)

In [7]:
n_components = 3
free_energy = flory_.free_energy.FloryHuggins(n_components, chis)
interaction = free_energy.interaction
entropy = free_energy.entropy
ensemble = flory_.CanonicalEnsemble(n_components, phi_global)
options = {
    "num_part": 16,
    "progress": False,
    "max_steps": 10000,  # disable progress bar, allow more steps
}

finder = flory_.CoexistingPhasesFinder(interaction, entropy, ensemble, **options)


In [8]:
phases = finder.run(progress=False)
vols = phases.volumes
concs = phases.fractions
vols, concs


(array([1.09276617, 1.43691589, 0.73111663, 0.61672514, 0.65925649,
        1.42183599, 0.91986161, 1.44220516, 0.74660872, 0.61944738,
        1.01436488, 1.39305461, 0.78934505, 1.18119735, 1.37044838,
        0.56485055]),
 array([[0.02758965, 0.07403521, 0.8983752 ],
        [0.89837398, 0.07403602, 0.02759011],
        [0.02758965, 0.07403521, 0.8983752 ],
        [0.08654541, 0.82690828, 0.08654615],
        [0.89837398, 0.07403602, 0.02759011],
        [0.02758965, 0.07403521, 0.8983752 ],
        [0.02758965, 0.07403521, 0.8983752 ],
        [0.02758965, 0.07403521, 0.8983752 ],
        [0.89837398, 0.07403602, 0.02759011],
        [0.08654541, 0.82690828, 0.08654615],
        [0.08654541, 0.82690828, 0.08654615],
        [0.02758965, 0.07403521, 0.8983752 ],
        [0.02758965, 0.07403521, 0.8983752 ],
        [0.08654541, 0.82690828, 0.08654615],
        [0.08654541, 0.82690828, 0.08654615],
        [0.02758965, 0.07403521, 0.8983752 ]]))

In [13]:
vols = phases.get_clusters().volumes
concs = phases.get_clusters().fractions

In [14]:
merged_compartments_list = [np.array([1, 2], dtype=np.int64)]
for merged_compartments in merged_compartments_list:
    phi_in_kunmerged, phi_in_kmerged, eta_merged, unmerged_compartment = mergers(concs, vols, chis, merged_compartments)

In [15]:
phi_global = np.array([phi_in_kunmerged, phi_in_kmerged])
vols = np.array([1-eta_merged, eta_merged])

In [16]:
phi_global

array([[0.8983752 , 0.08654615, 0.02759011],
       [0.04454157, 0.3567665 , 0.59869193]])

In [17]:
options = {
    "num_part": 2,
    "progress": False,
    "max_steps": 1000000000,  # disable progress bar, allow more steps
}
finder2 = flory_.CoexistingPhasesFinder(interaction, entropy, ensemble, **options)
finder2.reinitialize_from_phis(np.transpose(phi_global), vols)


In [18]:
phases = finder2.run(progress=False)
vols = phases.volumes
concs = phases.fractions
vols, concs


(array([0.32749014, 1.67250986]),
 array([[0.85573245, 0.11139882, 0.03286853],
        [0.07160261, 0.33692954, 0.59146789]]))